In [1]:
# In this notebook we will try to solve the prolem of overfitting by using L1 and L2 regularization technique.
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('D:\Coding\melbourne_housing_analisi\Melbourne_housing.csv')
df

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\39392\AppData\Local\Temp\ipykernel_25824\2578393646.py:1: SyntaxWarning: invalid escape sequence '\C'
  df = pd.read_csv('D:\Coding\melbourne_housing_analisi\Melbourne_housing.csv')
C:\Users\39392\AppData\Local\Temp\ipykernel_25824\2578393646.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\Coding\melbourne_housing_analisi\Melbourne_housing.csv')


,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longtitude,Regionname,Propertycount,ParkingArea,Price
0,Abbotsford,68 Studley St,2,h,SS,Jellis,3/9/2016,2.5,3067.0,2.0,...,126.0,inf,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0,Carport,NaN
1,Airport West,154 Halsey Rd,3,t,PI,Nelson,3/9/2016,13.5,3042.0,3.0,...,303.0,225,2016.0,Moonee Valley City Council,-37.71800,144.87800,Western Metropolitan,3464.0,Detached Garage,840000.0
2,Albert Park,105 Kerferd Rd,2,h,S,hockingstuart,3/9/2016,3.3,3206.0,2.0,...,120.0,82,1900.0,Port Phillip City Council,-37.84590,144.95550,Southern Metropolitan,3280.0,Attached Garage,1275000.0
3,Albert Park,85 Richardson St,2,h,S,Thomson,3/9/2016,3.3,3206.0,2.0,...,159.0,inf,NaN,Port Phillip City Council,-37.84500,144.95380,Southern Metropolitan,3280.0,Indoor,1455000.0
4,Alphington,30 Austin St,3,h,SN,McGrath,3/9/2016,6.4,3078.0,3.0,...,174.0,122,2003.0,Darebin City Council,-37.78180,145.01980,Northern Metropolitan,2211.0,Parkade,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Reservoir,18 Elinda Pl,3,u,SP,RW,30/09/2017,12.0,3073.0,3.0,...,NaN,105.0,1990.0,Darebin City Council,-37.69769,145.02332,Northern Metropolitan,21650.0,Parkade,475000.0
34853,Roxburgh Park,14 Stainsby Cr,4,h,S,Raine,30/09/2017,20.6,3064.0,4.0,...,NaN,225.0,1995.0,Hume City Council,-37.63665,144.92976,Northern Metropolitan,5833.0,Underground,591000.0
34854,Springvale South,8 Bellbird Ct,4,h,PI,Barry,30/09/2017,22.2,3172.0,4.0,...,534.0,152.0,1970.0,Greater Dandenong City Council,-37.97037,145.15449,South-Eastern Metropolitan,4054.0,Carport,NaN
34855,Springvale South,30 Waddington Cr,3,h,S,Harcourts,30/09/2017,22.2,3172.0,3.0,...,544.0,NaN,NaN,Greater Dandenong City Council,-37.97751,145.14813,South-Eastern Metropolitan,4054.0,Detached Garage,780500.0


In [3]:
df.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom             15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       994
YearBuilt          160
CouncilArea         33
Latitude         13402
Longtitude       14524
Regionname           8
Propertycount      342
ParkingArea          8
Price             2871
dtype: int64

In [4]:
df.shape

(34857, 22)

In [5]:
# now lets observe our data and try to drop some columns that are not very uselful in our data analysis. I am just dropping some columns
# like date, latitute etc that are not very meaningful for our analysis. 
columns_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 'Bedroom', 'Bathroom', 'Postcode', 'Address','Car', 'Price']
df_new = df[columns_to_use]
df_new

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Bedroom,Bathroom,Postcode,Address,Car,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.0,1.0,3067.0,68 Studley St,1.0,NaN
1,Airport West,3,t,PI,Nelson,Western Metropolitan,3464.0,3.0,2.0,3042.0,154 Halsey Rd,1.0,840000.0
2,Albert Park,2,h,S,hockingstuart,Southern Metropolitan,3280.0,2.0,1.0,3206.0,105 Kerferd Rd,0.0,1275000.0
3,Albert Park,2,h,S,Thomson,Southern Metropolitan,3280.0,2.0,1.0,3206.0,85 Richardson St,0.0,1455000.0
4,Alphington,3,h,SN,McGrath,Northern Metropolitan,2211.0,3.0,2.0,3078.0,30 Austin St,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Reservoir,3,u,SP,RW,Northern Metropolitan,21650.0,3.0,1.0,3073.0,18 Elinda Pl,1.0,475000.0
34853,Roxburgh Park,4,h,S,Raine,Northern Metropolitan,5833.0,4.0,2.0,3064.0,14 Stainsby Cr,2.0,591000.0
34854,Springvale South,4,h,PI,Barry,South-Eastern Metropolitan,4054.0,4.0,2.0,3172.0,8 Bellbird Ct,2.0,NaN
34855,Springvale South,3,h,S,Harcourts,South-Eastern Metropolitan,4054.0,3.0,2.0,3172.0,30 Waddington Cr,1.0,780500.0


In [6]:
df_new.shape

(34857, 13)

In [7]:
# now lets do some data cleaning
df_new.isna().sum()

Suburb              0
Rooms               0
Type                0
Method              0
SellerG             0
Regionname          0
Propertycount       3
Bedroom          8217
Bathroom         8226
Postcode            1
Address             0
Car              8728
Price            7610
dtype: int64

In [8]:
# so we have several columns with NaN values so we need to handle these columns. We can actually fill some of these column's NaN 
# values just by 0 and some other columns might need some other treatment based on their nature for example price.
# lets first handle the columns where we need to fill only 0.

columns_to_fill_0 = ['Car', 'Bathroom', 'Bedroom']
df_new[columns_to_fill_0] = df[columns_to_fill_0].fillna(0)
df_new = df_new.replace([np.inf, -np.inf], np.nan).dropna()
df_new.isna().sum()

C:\Users\39392\AppData\Local\Temp\ipykernel_25824\1343390596.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[columns_to_fill_0] = df[columns_to_fill_0].fillna(0)


Suburb           0
Rooms            0
Type             0
Method           0
SellerG          0
Regionname       0
Propertycount    0
Bedroom          0
Bathroom         0
Postcode         0
Address          0
Car              0
Price            0
dtype: int64

In [9]:
# Now lets fill the columns named landsize and building area with mean of the whole respective columns
df_new = df_new.replace([np.inf, -np.inf], np.nan).dropna()
# in the buildingArea column there are some infitly large valuea and the model was not training because of that that is why 
# I had to come back here and drop those inf values as well.


In [10]:
df_new.isna().sum()

Suburb           0
Rooms            0
Type             0
Method           0
SellerG          0
Regionname       0
Propertycount    0
Bedroom          0
Bathroom         0
Postcode         0
Address          0
Car              0
Price            0
dtype: int64

In [11]:
# now we are good to go with out cleaned data. Now we are going to make dummy variables for our whole dataset.
df_new = pd.get_dummies(df_new, drop_first=True) # it is a short cut to avoid dummy variable trap it is just dropping the main column whose dummies we have produced. 
df_new

,Rooms,Propertycount,Bedroom,Bathroom,Postcode,Car,Price,Suburb_Aberfeldie,Suburb_Airport West,Suburb_Albanvale,...,Address_9a Elizabeth St,Address_9a Field St,Address_9a Ivan Av,Address_9a Kennedy St,Address_9a Lilian Ct,Address_9a Russell St,Address_9a The Crossway,Address_9b Latham St,Address_9b Marquis Rd,Address_9b Stewart St
1,3,3464.0,3.0,2.0,3042.0,1.0,840000.0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,2,3280.0,2.0,1.0,3206.0,0.0,1275000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2,3280.0,2.0,1.0,3206.0,0.0,1455000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,4,2211.0,3.0,2.0,3078.0,4.0,2000000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,3,2211.0,3.0,2.0,3078.0,2.0,1110000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34851,3,11806.0,3.0,1.0,3174.0,6.0,627500.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34852,3,21650.0,3.0,1.0,3073.0,1.0,475000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34853,4,5833.0,4.0,2.0,3064.0,2.0,591000.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34855,3,4054.0,3.0,2.0,3172.0,1.0,780500.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
x = df_new.drop('Price', axis='columns')
y = df_new.Price

In [13]:
#rimozione outlier
numeric_cols = df.select_dtypes(include=["float", "int"]).columns.tolist()

for col in numeric_cols:
    Q1 = df[col].quantile(0.2)
    Q3 = df[col].quantile(0.8)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    df[col] = np.where(df[col].between(lower, upper), df[col], np.nan)

df = df.dropna()

In [15]:
from sklearn.discriminant_analysis import StandardScaler


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

MemoryError: Unable to allocate 5.43 GiB for an array with shape (26747, 27244) and data type float64

In [ ]:
# Now we can jump into our machine learning model and lets first use the train_test_split method
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, train_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [ ]:
model.score(x_test, y_test)
# Our model is much overfit with the training dataset that its accuracy in negative when we provide it with testing dataset. 

0.6234566030785157

In [ ]:
model.score(x_train, y_train) # at the same our model is performing very well with respect to the training datset

0.6929590554932192

In [ ]:
from sklearn.linear_model import Lasso


for a in [0.1, 1, 10, 50]:
    model = Lasso(alpha=a)
    model.fit(x_train, y_train)
    print(a, model.score(x_test, y_test))
    print(a, model.score(x_train, y_train))


c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.558e+14, tolerance: 2.341e+11
  model = cd_fast.enet_coordinate_descent(


0.1 0.6242076946790838
0.1 0.6929546092385062


c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.238e+14, tolerance: 2.341e+11
  model = cd_fast.enet_coordinate_descent(


1 0.6243078601096668
1 0.6929535325055567


c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.191e+14, tolerance: 2.341e+11
  model = cd_fast.enet_coordinate_descent(


10 0.6251166296515855
10 0.692938220334332
50 0.6270116223190811
50 0.692878442097215


c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.683e+13, tolerance: 2.341e+11
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Now we will use the L2 regularization tehnique
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.1)
ridge_model.fit(x_train, y_train)

,alpha,0.1
,fit_intercept,True
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


In [ ]:
ridge_model.score(x_test,y_test)
# after using L2 regularization our model is also much better but it seems that L1 regularization is slightly better then L2 in this case.

0.6235109435424937

In [ ]:
ridge_model.score(x_train,y_train)

0.6929590015836193

In [ ]:
from sklearn.linear_model import ElasticNet


ela = ElasticNet(alpha=0.1, l1_ratio=0.5)
ela.fit(x_train, y_train)
ela.score(x_test, y_test)


0.629427072060665